In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
Data =  pd.read_json('/content/drive/My Drive/Sarcasm Detection/Arabic News Headlines Dataset.json', orient = 'split')
Data.head()

,headlines,topic,length,is_sarcastic
0,طالب يصبح متعلّماً بعد دراسته وحدة عن القائد ف...,Society,12,1
1,سموحة يقلص الفارق ويسجل هدفا ثانيا في شباك الم...,Sports,11,0
2,ليتها الحدود – الولايات المتحدة تهدد بفرض عقوب...,Misc,17,1
3,وزيرة الهجرة تتواصل مع سفير مصر في نيوزيلندا ل...,None,14,0
4,ليتها الحدود – مصر تقرر إرسال أعضاء السائح الب...,Misc,12,1


In [0]:
X = Data['headlines']
Labels = Data['is_sarcastic']

In [5]:
import numpy as np
max_sequence_length = np.max(Data['length'])
print("max length is:", max_sequence_length)

max length is: 34


In [6]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 20595 unique tokens.


In [7]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (6669, 34)


In [8]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(Labels))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (6669, 2)


In [0]:
import pickle as p
fastText = p.load(open('/content/drive/My Drive/Sarcasm Detection/FastText_Wiki_Embeddings.p', 'rb'))

In [10]:
# prepare embedding matrix  (matrix of embeddings of the words in our dataset)
num_words = len(word_index) + 1
# initialize the np array
unKnown = list()
embedding_matrix = np.zeros((num_words, 300))
l=0
# find the embeddings from the pre trained embedding matrix of fast text wiki
for word, i in word_index.items():
    embedding_vector = fastText.get(word)
    if embedding_vector is not None:
        l+=1
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
      unKnown.append(word)
print("the number of words found in fast text embeddings =", l)
print("the number of words not found in fast text embeddings =", 20595- l)

the number of words found in fast text embeddings = 17289
the number of words not found in fast text embeddings = 3306


In [0]:
from keras.initializers import Constant
from keras.layers import Input, Embedding, Conv2D, GlobalMaxPooling2D, Dense, Dropout, LSTM, Bidirectional
from keras.models import Model

In [0]:
# shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


## split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [0]:
embedding_layer = Embedding(num_words,
                            300,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

In [0]:
# the model

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')             ## input layer
embedded_sequences = embedding_layer(sequence_input)                            ## embeddings layer
X = LSTM(128, return_sequences=True)(embedded_sequences)
X = Dropout(0.5)(X)
X = LSTM(128, return_sequences=False)(X)
X = Dropout(0.5)(X)
X = Dense(30, activation='relu')(X)                                      
X = Dropout(0.5)(X)                            
preds = Dense(2, activation='softmax')(X)                                       ## softmax layer

model = Model(sequence_input, preds)

In [0]:
rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9
                                   , epsilon=None, decay=0.3)



In [22]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = rmsprop,
              metrics = ['acc'])

model.fit(x_train, y_train, 
          validation_split = 0.2, 
          batch_size = 32, 
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.6145 - acc: 0.6886 - val_loss: 0.5345 - val_acc: 0.8232
Epoch 2/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.5205 - acc: 0.7903 - val_loss: 0.4683 - val_acc: 0.8392
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.4864 - acc: 0.8015 - val_loss: 0.4324 - val_acc: 0.8472
Epoch 4/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.4597 - acc: 0.8138 - val_loss: 0.4112 - val_acc: 0.8551
Epoch 5/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.4460 - acc: 0.8143 - val_loss: 0.3972 - val_acc: 0.8571
Epoch 6/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.4356 - acc: 0.8185 - val_loss: 0.3868 - val_acc: 0.8591
Epoch 7/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.4259 - acc: 0.8253 - val_loss: 0.3790 - val_acc: 0.8611
Epoch 8

In [0]:
from keras import backend as K

def recall(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall


      
def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
      

def f1(y_true, y_pred):
    
    Precision = precision(y_true, y_pred)
    Recall = recall(y_true, y_pred)
    f1 = 2*((Precision*Recall)/(Precision+Recall+K.epsilon()))
    
    return f1



In [24]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = rmsprop,
              metrics = [precision])
model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, prec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', prec)


Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.3626 - precision: 0.8493 - val_loss: 0.3234 - val_precision: 0.8741
Epoch 2/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3613 - precision: 0.8480 - val_loss: 0.3230 - val_precision: 0.8721
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3689 - precision: 0.8470 - val_loss: 0.3227 - val_precision: 0.8741
Epoch 4/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3595 - precision: 0.8473 - val_loss: 0.3224 - val_precision: 0.8741
Epoch 5/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3646 - precision: 0.8520 - val_loss: 0.3221 - val_precision: 0.8751
Epoch 6/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3592 - precision: 0.8535 - val_loss: 0.3218 - val_precision: 0.8751
Epoch 7/50
4001/4001 [==============================] - 14s 4ms/step -

In [25]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = rmsprop,
              metrics = [recall])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, rec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', rec)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.3549 - recall: 0.8535 - val_loss: 0.3131 - val_recall: 0.8781
Epoch 2/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3510 - recall: 0.8510 - val_loss: 0.3130 - val_recall: 0.8781
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3528 - recall: 0.8563 - val_loss: 0.3128 - val_recall: 0.8771
Epoch 4/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3502 - recall: 0.8560 - val_loss: 0.3128 - val_recall: 0.8811
Epoch 5/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3557 - recall: 0.8525 - val_loss: 0.3127 - val_recall: 0.8811
Epoch 6/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.3516 - recall: 0.8545 - val_loss: 0.3125 - val_recall: 0.8811
Epoch 7/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3529 - recall: 0.8535 - val

In [26]:
model.compile(loss = 'categorical_crossentropy',
          optimizer = rmsprop, 
          metrics = [f1])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.3433 - f1: 0.8623 - val_loss: 0.3078 - val_f1: 0.8821
Epoch 2/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3521 - f1: 0.8535 - val_loss: 0.3077 - val_f1: 0.8821
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3459 - f1: 0.8548 - val_loss: 0.3076 - val_f1: 0.8821
Epoch 4/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.3498 - f1: 0.8595 - val_loss: 0.3075 - val_f1: 0.8821
Epoch 5/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3456 - f1: 0.8578 - val_loss: 0.3074 - val_f1: 0.8821
Epoch 6/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3371 - f1: 0.8550 - val_loss: 0.3074 - val_f1: 0.8821
Epoch 7/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.3422 - f1: 0.8583 - val_loss: 0.3073 - val_f1: 0.8821
Epoch 8/50
4001/4001 

In [0]:
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.25, amsgrad=False)

In [31]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = ['acc'])

model.fit(x_train, y_train, 
          validation_split = 0.2, 
          batch_size = 32, 
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.2638 - acc: 0.8965 - val_loss: 0.2451 - val_acc: 0.9041
Epoch 2/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2360 - acc: 0.9098 - val_loss: 0.2421 - val_acc: 0.9041
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2343 - acc: 0.9095 - val_loss: 0.2408 - val_acc: 0.9051
Epoch 4/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2316 - acc: 0.9113 - val_loss: 0.2410 - val_acc: 0.9041
Epoch 5/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2259 - acc: 0.9153 - val_loss: 0.2403 - val_acc: 0.9041
Epoch 6/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2229 - acc: 0.9145 - val_loss: 0.2401 - val_acc: 0.9041
Epoch 7/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2213 - acc: 0.9160 - val_loss: 0.2395 - val_acc: 0.9041
Epoch 8

In [32]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [precision])
model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, prec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', prec)


Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.2114 - precision: 0.9200 - val_loss: 0.2372 - val_precision: 0.9041
Epoch 2/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2113 - precision: 0.9208 - val_loss: 0.2372 - val_precision: 0.9051
Epoch 3/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2136 - precision: 0.9180 - val_loss: 0.2372 - val_precision: 0.9051
Epoch 4/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2025 - precision: 0.9213 - val_loss: 0.2371 - val_precision: 0.9051
Epoch 5/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2053 - precision: 0.9220 - val_loss: 0.2372 - val_precision: 0.9051
Epoch 6/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2097 - precision: 0.9215 - val_loss: 0.2371 - val_precision: 0.9051
Epoch 7/50
4001/4001 [==============================] - 13s 3ms/step -

In [33]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [recall])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, rec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', rec)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 18s 4ms/step - loss: 0.2045 - recall: 0.9213 - val_loss: 0.2368 - val_recall: 0.9051
Epoch 2/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2088 - recall: 0.9230 - val_loss: 0.2368 - val_recall: 0.9051
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2022 - recall: 0.9258 - val_loss: 0.2368 - val_recall: 0.9051
Epoch 4/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2089 - recall: 0.9180 - val_loss: 0.2368 - val_recall: 0.9051
Epoch 5/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2055 - recall: 0.9180 - val_loss: 0.2368 - val_recall: 0.9051
Epoch 6/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2076 - recall: 0.9200 - val_loss: 0.2367 - val_recall: 0.9051
Epoch 7/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2035 - recall: 0.9213 - val

In [34]:
model.compile(loss = 'categorical_crossentropy',
          optimizer = adam, 
          metrics = [f1])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 17s 4ms/step - loss: 0.2055 - f1: 0.9220 - val_loss: 0.2366 - val_f1: 0.9051
Epoch 2/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2018 - f1: 0.9215 - val_loss: 0.2366 - val_f1: 0.9051
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2030 - f1: 0.9263 - val_loss: 0.2366 - val_f1: 0.9051
Epoch 4/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2061 - f1: 0.9180 - val_loss: 0.2366 - val_f1: 0.9051
Epoch 5/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2028 - f1: 0.9203 - val_loss: 0.2366 - val_f1: 0.9051
Epoch 6/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.1986 - f1: 0.9208 - val_loss: 0.2367 - val_f1: 0.9051
Epoch 7/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2003 - f1: 0.9235 - val_loss: 0.2366 - val_f1: 0.9051
Epoch 8/50
4001/4001 

In [0]:
model.save_weights('/content/drive/My Drive/Sarcasm Detection/Plots/LSTM_optimized_Weights.h5')